In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from __future__ import print_function
import PIL.Image as Image
import matplotlib.pyplot as plt
import copy
import numpy as np

use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# desired size of the output image
# gpu 512 이상 시 out of memory error 발생
imsize = 512 if use_cuda else 128

transform = transforms.Compose([transforms.ToTensor()])

def image_loader(image_name, transform=None, max_size=None, shape=None):
    image = Image.open(image_name)
    
    if max_size is not None:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.ANTIALIAS)
    
    if shape is not None:
        image = image.resize(shape, Image.LANCZOS)
    
    if transform is not None:
        image = transform(image)
        
    image = Variable(image)
    image = image.unsqueeze(0)
    
    return image.type(dtype)

In [3]:
def mask_content(content_img, generated_img, mask_img):
    width = mask_img.size(2)
    height = mask_img.size(3)
    mask_img = mask_img.data.cpu().numpy()
    content_img = content_img.data
    
    for i in range(width):
        for j in range(height):
            if mask_img[0, :, i, j].all() == 0:
                generated_img[0, :, i, j] = content_img[0, :, i, j]
    
    return generated_img

In [4]:
class ContentLoss(nn.Module):

    def __init__(self, target, weight):
        super(ContentLoss, self).__init__()
        self.target = target.detach() * weight
        self.weight = weight
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.loss = self.criterion(input * self.weight, self.target)
        self.output = input
        return self.output

    def backward(self, retain_graph=True):
        self.loss.backward(retain_graph=retain_graph)
        return self.loss

In [5]:
class GramMatrix(nn.Module):

    def forward(self, input):
        a, b, c, d = input.size()
        features = input.view(a * b, c * d)
        G = torch.mm(features, features.t())
        return G.div(a * b * c * d)

In [6]:
class StyleLoss(nn.Module):

    def __init__(self, target, weight):
        super(StyleLoss, self).__init__()
        self.target = target.detach() * weight
        self.weight = weight
        self.gram = GramMatrix()
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.output = input.clone()
        self.G = self.gram(input)
        self.G.mul_(self.weight)
        self.loss = self.criterion(self.G, self.target)
        return self.output

    def backward(self, retain_graph=True):
        self.loss.backward(retain_graph=retain_graph)
        return self.loss

In [7]:
# VGG19를 Transfer Learning에 사용
cnn = models.vgg19(pretrained=True).features

# move it to the GPU if possible:
if use_cuda:
    cnn = cnn.cuda()

In [8]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']


def get_style_model_and_losses(cnn, style_img, content_img,
                               style_weight=1000, content_weight=1,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    model = nn.Sequential()  # the new Sequential module network
    gram = GramMatrix()  # we need a gram module in order to compute style targets

    # move these modules to the GPU if possible:
    if use_cuda:
        model = model.cuda()
        gram = gram.cuda()

    i = 1
    for layer in list(cnn):
        if isinstance(layer, nn.Conv2d):
            name = "conv_" + str(i)
            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).clone()
                content_loss = ContentLoss(target, content_weight)
                model.add_module("content_loss_" + str(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).clone()
                target_feature_gram = gram(target_feature)
                style_loss = StyleLoss(target_feature_gram, style_weight)
                model.add_module("style_loss_" + str(i), style_loss)
                style_losses.append(style_loss)

        if isinstance(layer, nn.ReLU):
            name = "relu_" + str(i)
            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).clone()
                content_loss = ContentLoss(target, content_weight)
                model.add_module("content_loss_" + str(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).clone()
                target_feature_gram = gram(target_feature)
                style_loss = StyleLoss(target_feature_gram, style_weight)
                model.add_module("style_loss_" + str(i), style_loss)
                style_losses.append(style_loss)

            i += 1

        if isinstance(layer, nn.MaxPool2d):
            name = "pool_" + str(i)
            model.add_module(name, layer)  # ***

    return model, style_losses, content_losses

In [9]:
def get_input_param_optimizer(input_img):
    input_param = nn.Parameter(input_img.data)
    optimizer = optim.LBFGS([input_param])
    return input_param, optimizer

In [10]:
# step : 300
# style weight = 1000 / content weight = 1
def train_style_transfer(cnn, content_img, style_img, input_img, num_steps=300,
                       style_weight=1000, content_weight=1):
    print('Starting style transfer!!')
    model, style_losses, content_losses = get_style_model_and_losses(cnn, style_img, content_img, style_weight, content_weight)
    input_param, optimizer = get_input_param_optimizer(input_img)
    
    run = [0]
    while run[0] <= num_steps:
        def steps():
            input_param.data.clamp_(0, 1)
            optimizer.zero_grad()
            model(input_param)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.backward()
            for cl in content_losses:
                content_score += cl.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}: Style Loss : {:4f} Content Loss: {:4f}".format(run, style_score.data[0], content_score.data[0]))

            return style_score + content_score

        optimizer.step(steps)

    # a last correction...
    input_param.data.clamp_(0, 1)

    return input_param.data

In [14]:
# Style or Content or Mask image 변경 시 여기서 변경
content_img = image_loader("images/Content/content.jpg", transform, max_size=imsize)
style_img = image_loader("images/Style/style.jpg", transform, shape=[content_img.size(2), content_img.size(3)])
input_img = content_img.clone()
mask_img = image_loader("images/Mask/mask.jpg", transform, shape=[content_img.size(3), content_img.size(2)])

In [15]:
# Training
output = train_style_transfer(cnn, content_img, style_img, input_img)

Starting style transfer!!
run [50]: Style Loss : 0.535881 Content Loss: 1.586229
run [100]: Style Loss : 0.245845 Content Loss: 1.358469
run [150]: Style Loss : 0.188381 Content Loss: 1.303700
run [200]: Style Loss : 0.165162 Content Loss: 1.276881
run [250]: Style Loss : 0.157607 Content Loss: 1.261195
run [300]: Style Loss : 0.157834 Content Loss: 1.248573


In [16]:
torchvision.utils.save_image(output, 'images/Output/ouput_single.jpg')
output = mask_content(content_img, output, mask_img)
torchvision.utils.save_image(output, 'images/Output/ouput_masked.jpg')